# Scaffold
Prototype Thudbot in jupyter, then figure out how to convert to a .py   


Naming this 00_ so that it will be the first notebook every time I start a new session.

#### Step 1 Source all the keys and environment variables

In [1]:
### API key management

### Reminder: Place .env file inside the root of the project folder so when calling the below from inside the notebook it should find the .env fule and load it inside the notebook environment
### PLEASE ADD THIS `.env` FILE TO YOUR PROJECT'S `.gitignore` file before committing and pushing the changes to your remote repo, as it contains API Keys and Secrets in it

import os
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env")

# --- Verify API Keys ---
print("--- API Key Status ---")
print(f"OPENAI_API_KEY loaded: {'OPENAI_API_KEY' in os.environ}")
print(f"LANGCHAIN_API_KEY loaded: {'LANGCHAIN_API_KEY' in os.environ}")
print(f"TAVILY_API_KEY loaded: {'TAVILY_API_KEY' in os.environ}")
print(f"RAGAS_API_KEY loaded: {'RAGAS_API_KEY' in os.environ}")
print(f"ANTHROPIC_API_KEY loaded: {'ANTHROPIC_API_KEY' in os.environ}")
print(f"COHERE_API_KEY loaded: {'COHERE_API_KEY' in os.environ}")

# --- Verify General Settings ---
print("\n--- Project Settings Status ---")
print(f"DEBUG mode enabled: {os.environ.get('DEBUG') == 'True'}")
print(f"LangSmith Tracing V2 enabled: {os.environ.get('LANGCHAIN_TRACING_V2') == 'true'}")
print(f"LangChain Project Base: {os.environ.get('LANGCHAIN_PROJECT_BASE')}")

--- API Key Status ---
OPENAI_API_KEY loaded: True
LANGCHAIN_API_KEY loaded: True
TAVILY_API_KEY loaded: True
RAGAS_API_KEY loaded: False
ANTHROPIC_API_KEY loaded: True
COHERE_API_KEY loaded: True

--- Project Settings Status ---
DEBUG mode enabled: True
LangSmith Tracing V2 enabled: True
LangChain Project Base: THUDBOT-CC


including nltk, because it worked before

In [26]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/family/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/family/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Step 2: Data Collection and Preparation

My data is CSV structured, so using code from HW9

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

loader = CSVLoader(
    file_path=f"./data/Thudbot_Hint_Data_1.csv",
    metadata_columns=[
        "question",
        "hint_level",
        "character",
        "speaker",
        "narrative_context",
        "planet",
        "location",
        "category",
        "tone",
        "follow_up_hint_id",
        "answer_keywords",
        "tags"
    ]
)

hint_data = loader.load()

# No need to overwrite page_content; not doing custom transformation
print(hint_data[0].page_content)     # This will already be the hint_text
print(hint_data[0].metadata)         # This will show all the metadata fields


question_id: TSB-001
hint_text: Press the escape key to exit the opening animations
puzzle_name: 
source: self
{'source': './data/Thudbot_Hint_Data_1.csv', 'row': 0, 'question': 'How do I stop the opening movie', 'hint_level': '1', 'character': 'Player', 'speaker': '', 'narrative_context': 'Meta', 'planet': '', 'location': '', 'category': 'Meta', 'tone': '', 'follow_up_hint_id': '', 'answer_keywords': '', 'tags': ''}


## Step 3: Setting up QDrant! (from HW9)

Now that we have our documents, let's create a QDrant VectorStore with the collection name "ThudbotHints".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

 

In [24]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents=hint_data,
    embedding=embeddings,
    location=":memory:",
    collection_name="Thudbot_Hints"
)

#### Step 3: Golden Data Set SDG from RAGAS

Adapted from HW 7 & 9


In [27]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/family/Library/Mobile Documents/com~apple~CloudDocs/AppDev/thudbot/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/family/Library/Mobile Documents/com~apple~CloudDocs/AppDev/thudbot/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/family/Library/Mobile Documents/com~apple~CloudDocs/AppDev/thudbot/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


#### Group data by narrative context

the data is too granular for Ragas, I got this error:

ValueError: Documents appears to be too short (ie 100 tokens or less). Please provide longer documents

so,  I will to group it and 
Create a new list called  ```merged_docs``` , just to feed to ragas, but not for retrieval

In [33]:
from langchain.schema import Document
from collections import defaultdict

# Group hints by narrative context
grouped = defaultdict(list)

for doc in hint_data:
    key = doc.metadata.get("narrative_context", "unknown")
    grouped[key].append(doc.page_content)

# Create longer Documents for SDG
merged_docs = [
    Document(
        page_content="\n".join(hints),
        metadata={"narrative_context": context}
    )
    for context, hints in grouped.items()
]

# Optional: preview length
for doc in merged_docs:
    print(f"{doc.metadata['narrative_context']}: {len(doc.page_content.split())} words")


Meta: 505 words
Bar: 562 words
Background: 75 words
Thud Flashback: 94 words
Fleebix Flashback: 1725 words


Need to select a random subset, because the data is grouped

In [35]:
from ragas.testset import TestsetGenerator
import random

sampled_docs = random.sample(merged_docs, 5)
generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
golden_dataset = generator.generate_with_langchain_docs(sampled_docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/3 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/5 [00:00<?, ?it/s]

ERROR:ragas.testset.transforms.engine:unable to apply transformation: 'headlines' property not found in this node
ERROR:ragas.testset.transforms.engine:unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/4 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/5 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/14 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [36]:
golden_dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,"In the context of the game, what is the signif...",[question_id: TSB-045 hint_text: To get to the...,"The hint_text indicates that in Yzore, you nee...",single_hop_specifc_query_synthesizer
1,TSB-041 what do I do?,[question_id: TSB-041 hint_text: You can't do ...,"You can't do much on your own as Fleebix, beca...",single_hop_specifc_query_synthesizer
2,What UHS mean in the game?,[look in the small package when you were insid...,UHS is referenced as the source for puzzles an...,single_hop_specifc_query_synthesizer
3,WhaT is TSB-027?,[question_id: TSB-019 hint_text: Look around i...,Thud is not very bright.,single_hop_specifc_query_synthesizer
4,H0w do the residue printer and fingerprint ana...,[<1-hop>\n\nquestion_id: TSB-019 hint_text: Lo...,The residue printer can tell you who may have ...,multi_hop_abstract_query_synthesizer
5,how use tokens for bus and find object like do...,[<1-hop>\n\nquestion_id: TSB-045 hint_text: To...,"In the context, you need a token to get on the...",multi_hop_abstract_query_synthesizer
6,How can you effectively utilize navigation and...,[<1-hop>\n\nquestion_id: TSB-045 hint_text: To...,To effectively navigate the Simulator and expl...,multi_hop_abstract_query_synthesizer
7,How Fleebix rely on Thud and follow clues to g...,[<1-hop>\n\nquestion_id: TSB-041 hint_text: Yo...,Fleebix can't do much alone and needs Thud's h...,multi_hop_abstract_query_synthesizer
8,how to get to Quantelope Lodge with Thud and F...,[<1-hop>\n\nquestion_id: TSB-041 hint_text: Yo...,"First, you need to find Thud and Fleebix and g...",multi_hop_specific_query_synthesizer
9,How do Fleebix and Thud get to the Quantelope ...,[<1-hop>\n\nquestion_id: TSB-041 hint_text: Yo...,Fleebix and Thud need to reach the Quantelope ...,multi_hop_specific_query_synthesizer


saving the RAGAS golden data set for later

In [37]:
import json

with open("data/goldendataset.json", "w") as f:
    json.dump([sample.model_dump() for sample in golden_dataset], f, indent=2)


In [38]:
## Export Golden Dataset to Human-Readable Formats

import json
import pandas as pd

# Load the golden dataset
with open('./data/goldendataset.json', 'r') as f:
    golden_data = json.load(f)

# Extract the key information into a clean DataFrame
readable_data = []
for i, item in enumerate(golden_data):
    eval_sample = item['eval_sample']
    
    # Clean up the reference contexts (truncate if too long)
    contexts = eval_sample.get('reference_contexts', [])
    contexts_preview = str(contexts[0])[:200] + "..." if contexts else "No context"
    
    readable_data.append({
        'Question_ID': f"Q{i+1:02d}",
        'User_Question': eval_sample.get('user_input', ''),
        'Expected_Answer': eval_sample.get('reference', ''),
        'Synthesizer_Type': item.get('synthesizer_name', ''),
        'Context_Preview': contexts_preview,
        'Full_Context_Length': len(str(contexts)) if contexts else 0
    })

# Create DataFrame
df = pd.DataFrame(readable_data)

# Export to CSV
df.to_csv('./data/golden_dataset_readable.csv', index=False)

# Display preview
print("Golden Dataset Preview:")
print("=" * 60)
for _, row in df.head(3).iterrows():
    print(f"Question {row['Question_ID']}: {row['User_Question']}")
    print(f"Expected: {row['Expected_Answer']}")
    print(f"Synthesizer: {row['Synthesizer_Type']}")
    print("-" * 40)

print(f"\n✅ Exported {len(df)} questions to: ./data/golden_dataset_readable.csv")


Golden Dataset Preview:
Question Q01: In the context of the game, what is the significance of the Yzore in reaching the Quantelope Lodge?
Expected: The hint_text indicates that in Yzore, you need a token to get on a bus, and there's a token in the cup. You can ask Thud to take the token from the cup, which is part of the process to reach the Quantelope Lodge.
Synthesizer: single_hop_specifc_query_synthesizer
----------------------------------------
Question Q02: TSB-041 what do I do?
Expected: You can't do much on your own as Fleebix, because you are in a jar. To progress, you might need to rely on Thud to help you with many tasks, such as finding Thud or interacting with objects like the door or the cup, to get to Quantelope Lodge.
Synthesizer: single_hop_specifc_query_synthesizer
----------------------------------------
Question Q03: What UHS mean in the game?
Expected: UHS is referenced as the source for puzzles and hints in the game, such as Clocktower and Navigation Simulator, but

In [39]:
## Option 2: Create Detailed Markdown Report

def create_markdown_report(golden_data, output_file='./data/golden_dataset_report.md'):
    """Create a detailed, human-readable markdown report of the golden dataset."""
    
    with open(output_file, 'w') as f:
        f.write("# Thudbot Golden Dataset Report\n\n")
        f.write("Generated synthetic test questions and answers for RAGAS evaluation.\n\n")
        f.write(f"**Total Questions:** {len(golden_data)}\n\n")
        f.write("---\n\n")
        
        for i, item in enumerate(golden_data, 1):
            eval_sample = item['eval_sample']
            
            f.write(f"## Question {i:02d}\n\n")
            f.write(f"**User Input:** {eval_sample.get('user_input', 'N/A')}\n\n")
            f.write(f"**Expected Answer:**\n{eval_sample.get('reference', 'N/A')}\n\n")
            f.write(f"**Synthesizer:** `{item.get('synthesizer_name', 'unknown')}`\n\n")
            
            # Add context information
            contexts = eval_sample.get('reference_contexts', [])
            if contexts:
                f.write("**Reference Context Preview:**\n")
                context_preview = str(contexts[0])[:300] + "..." if len(str(contexts[0])) > 300 else str(contexts[0])
                f.write(f"```\n{context_preview}\n```\n\n")
            
            f.write("---\n\n")
    
    print(f"✅ Created detailed report: {output_file}")

# Generate the markdown report
create_markdown_report(golden_data)


✅ Created detailed report: ./data/golden_dataset_report.md


## Step 4: RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [15]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})